In [1]:
2 + 2

4

In [2]:
! pip list

Package                       Version
----------------------------- ---------
alembic                       1.7.4
altair                        4.1.0
anyio                         3.3.2
appdirs                       1.4.4
argon2-cffi                   20.1.0
async-generator               1.10
attrs                         21.2.0
Babel                         2.9.1
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
beautifulsoup4                4.10.0
bleach                        4.1.0
blinker                       1.4
bokeh                         2.4.0
Bottleneck                    1.3.2
brotlipy                      0.7.0
cached-property               1.5.2
certifi                       2021.5.30
certipy                       0.1.3
cffi                          1.14.6
chardet                       4.0.0
charset-normalizer            2.0.0
click                         8.0.1
cloudpickle                   2.0.0
colorama                      0.4.4
conda            

In [3]:
! pip install fastavro

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = (SparkSession
         .builder
         .master('local')
         .appName('person-data-analyzer')
         .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2")
         .getOrCreate())
spark.sparkContext.setLogLevel("OFF")
sc = spark.sparkContext

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ee65c147-b5a9-408d-88c8-e543863791be;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 599ms :: artifacts dl 13ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central

In [6]:
spark.sparkContext.setLogLevel("OFF")

In [7]:
df = (spark
  .read
  .format("kafka")
  .option("kafka.bootstrap.servers", "kafka-server:9092")
  .option("subscribe", "persons-json")
  .option("startingOffsets", "earliest") 
  .load())

In [8]:
df.count()

100000

In [9]:
from pyspark.sql.types import StringType

df1 = (df
    .withColumn("key", df["key"].cast(StringType()))
    .withColumn("value", df["value"].cast(StringType())))

In [10]:
from pyspark.sql.functions import from_json

In [11]:
from pyspark.sql.types import StructType, StructField, LongType, TimestampType

In [12]:
person_json_schema = StructType(
  [
      StructField("id", LongType(), True),
      StructField("firstname", StringType(), True),
      StructField("lastname", StringType(), True),
      StructField("email", StringType(), True),
      StructField("profession", StringType(), True),
      StructField("city", StringType(), True),
      StructField("country", StringType(), True),
      StructField("random_date", TimestampType(), True)
  ]
)

In [13]:
person_df_json = df1.withColumn("schema_value", from_json("value", person_json_schema))

In [14]:
person_df_json.show(5)

+----+--------------------+------------+---------+------+--------------------+-------------+--------------------+
| key|               value|       topic|partition|offset|           timestamp|timestampType|        schema_value|
+----+--------------------+------------+---------+------+--------------------+-------------+--------------------+
|null|{"id": 1, "firstn...|persons-json|        0|     0|2021-10-09 14:22:...|            0|{1, Cyndie, Chain...|
|null|{"id": 2, "firstn...|persons-json|        0|     1|2021-10-09 14:22:...|            0|{2, Edyth, Bultma...|
|null|{"id": 3, "firstn...|persons-json|        0|     2| 2021-10-09 14:22:45|            0|{3, Mignon, Martg...|
|null|{"id": 4, "firstn...|persons-json|        0|     3|2021-10-09 14:22:...|            0|{4, Allis, Aberno...|
|null|{"id": 5, "firstn...|persons-json|        0|     4|2021-10-09 14:22:...|            0|{5, Morganica, Qu...|
+----+--------------------+------------+---------+------+--------------------+----------

In [15]:
from pyspark.sql.functions import col

person_df_json_formatted = (
   person_df_json.select(
     col("timestamp").alias("event_timestamp"),
     col("schema_value.id").alias("id"),
     col("schema_value.firstname").alias("first_name"),
     col("schema_value.lastname").alias("last_name"),
     col("schema_value.email").alias("email"),
     col("schema_value.profession").alias("profession"),
     col("schema_value.city").alias("city"),
     col("schema_value.country").alias("country"),
     col("schema_value.random_date").alias("random_date")  
   )
)

In [16]:
person_df_json_formatted.show(5)

+--------------------+---+----------+-----------+--------------------+-----------+----------+--------------------+-----------+
|     event_timestamp| id|first_name|  last_name|               email| profession|      city|             country|random_date|
+--------------------+---+----------+-----------+--------------------+-----------+----------+--------------------+-----------+
|2021-10-09 14:22:...|  1|    Cyndie|     Chaing|Cyndie.Chaing@yop...|     worker|     Lhasa|              Mexico|       null|
|2021-10-09 14:22:...|  2|     Edyth|    Bultman|Edyth.Bultman@yop...|firefighter|   Cologne|         New Zealand|       null|
| 2021-10-09 14:22:45|  3|    Mignon|Martguerita|Mignon.Martguerit...|     worker|Concepción|       Bouvet Island|       null|
|2021-10-09 14:22:...|  4|     Allis|    Abernon|Allis.Abernon@yop...|  developer|   Geelong|         Isle of Man|       null|
|2021-10-09 14:22:...|  5| Morganica|      Quent|Morganica.Quent@y...|     worker|   Pattaya|Saint Pierre and .

In [17]:
from pyspark.sql.functions import count

person_df_json_counted = (
  person_df_json_formatted.groupby(col("profession"))
                          .agg(count(col("profession")).alias("profession_count")) 
)

In [18]:
person_df_json_counted.show()

+--------------+----------------+
|    profession|profession_count|
+--------------+----------------+
|     developer|           19878|
|   firefighter|           20048|
|police officer|           19905|
|        worker|           20154|
|        doctor|           20015|
+--------------+----------------+



In [19]:
import io
import fastavro

In [20]:
def deserialize_avro(serialized_msg):
    bytes_io = io.BytesIO(serialized_msg)
    bytes_io.seek(0)
    avro_schema = {
        'type': 'record',
        'name': 'PersonRecord',
        'fields': [
            {'name': 'id', 'type': 'long'},
            {'name': 'firstname', 'type': 'string'},
            {'name': 'lastname', 'type': 'string'},
            {'name': 'email', 'type': 'string'},
            {'name': 'profession', 'type': 'string'},
            {'name': 'city', 'type': 'string'},
            {'name': 'country', 'type': 'string'},
            {'name': 'random_date', 'type': {'type': 'long', 'logicalType': 'timestamp-millis'}}
        ],
        'doc': 'Definition of Person record model',
        'namespace': 'Person.v1',
        'aliases': ['person-v1', 'person-record']
    }

    deserialized_msg = fastavro.schemaless_reader(bytes_io, avro_schema)

    return (    deserialized_msg["id"],
                deserialized_msg["firstname"],
                deserialized_msg["lastname"],
                deserialized_msg["email"],
                deserialized_msg["profession"],
                deserialized_msg["city"],
                deserialized_msg["country"],
                deserialized_msg["random_date"],
            )

In [21]:
df_1 = (spark
  .read
  .format("kafka")
  .option("kafka.bootstrap.servers", "kafka-server:9092")
  .option("subscribe", "persons-avro")
  .option("startingOffsets", "earliest") 
  .load())

In [22]:
df_1.count()

100000

In [23]:
from pyspark.sql.functions import udf
from pyspark.sql.types import TimestampType

In [24]:
person_avro_schema = StructType(
  [
      StructField("id", LongType(), True),
      StructField("firstname", StringType(), True),
      StructField("lastname", StringType(), True),
      StructField("email", StringType(), True),
      StructField("profession", StringType(), True),
      StructField("city", StringType(), True),
      StructField("country", StringType(), True),
      StructField("random_date", TimestampType(), True)
  ]
)

In [25]:
avro_deserialize_udf = udf(deserialize_avro, returnType=person_avro_schema)

In [26]:
persons_avro_df = df_1.withColumn("avro_value", avro_deserialize_udf(col("value")))

In [27]:
persons_avro_df.show(5)

+----+--------------------+------------+---------+------+--------------------+-------------+--------------------+
| key|               value|       topic|partition|offset|           timestamp|timestampType|          avro_value|
+----+--------------------+------------+---------+------+--------------------+-------------+--------------------+
|null|[02 0C 43 79 6E 6...|persons-avro|        0|     0|2021-10-09 14:22:...|            0|{1, Cyndie, Chain...|
|null|[04 0A 45 64 79 7...|persons-avro|        0|     1|2021-10-09 14:22:...|            0|{2, Edyth, Bultma...|
|null|[06 0C 4D 69 67 6...|persons-avro|        0|     2|2021-10-09 14:22:...|            0|{3, Mignon, Martg...|
|null|[08 0A 41 6C 6C 6...|persons-avro|        0|     3|2021-10-09 14:22:...|            0|{4, Allis, Aberno...|
|null|[0A 12 4D 6F 72 6...|persons-avro|        0|     4|2021-10-09 14:22:...|            0|{5, Morganica, Qu...|
+----+--------------------+------------+---------+------+--------------------+----------

In [28]:
persons_avro_df_formatted = (
   persons_avro_df.select(
     col("timestamp").alias("event_timestamp"),
     col("avro_value.id").alias("id"),
     col("avro_value.firstname").alias("first_name"),
     col("avro_value.lastname").alias("last_name"),
     col("avro_value.email").alias("email"),
     col("avro_value.profession").alias("profession"),
     col("avro_value.city").alias("city"),
     col("avro_value.country").alias("country"),
     col("avro_value.random_date").alias("random_date")  
   )
)

In [29]:
persons_avro_df_formatted.show(5)

+--------------------+---+----------+-----------+--------------------+-----------+----------+--------------------+-------------------+
|     event_timestamp| id|first_name|  last_name|               email| profession|      city|             country|        random_date|
+--------------------+---+----------+-----------+--------------------+-----------+----------+--------------------+-------------------+
|2021-10-09 14:22:...|  1|    Cyndie|     Chaing|Cyndie.Chaing@yop...|     worker|     Lhasa|              Mexico|1922-03-19 21:00:00|
|2021-10-09 14:22:...|  2|     Edyth|    Bultman|Edyth.Bultman@yop...|firefighter|   Cologne|         New Zealand|1907-04-11 21:29:43|
|2021-10-09 14:22:...|  3|    Mignon|Martguerita|Mignon.Martguerit...|     worker|Concepción|       Bouvet Island|1957-09-27 21:00:00|
|2021-10-09 14:22:...|  4|     Allis|    Abernon|Allis.Abernon@yop...|  developer|   Geelong|         Isle of Man|1968-08-31 21:00:00|
|2021-10-09 14:22:...|  5| Morganica|      Quent|Morgan

In [31]:
persons_avro_df_counted = (
  persons_avro_df_formatted.groupby(col("profession"))
                          .agg(count(col("profession")).alias("profession_count")) 
)

In [32]:
persons_avro_df_counted.show()

+--------------+----------------+
|    profession|profession_count|
+--------------+----------------+
|     developer|           19878|
|   firefighter|           20048|
|police officer|           19905|
|        worker|           20154|
|        doctor|           20015|
+--------------+----------------+

